In [1]:
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')
test_iter = IMDB(split='test')

In [2]:
import random
random.seed(6)

# train_iter를 리스트 타입으로 변경
train_lists = list(train_iter)
test_lists = list(test_iter)

# 각기 1000개씩 랜덤 샘플링
train_lists_small = random.sample(train_lists, 1000)
test_lists_small = random.sample(test_lists, 1000)

# 각 변수에 담긴 인덱스 0에 해당하는 원소, 즉 첫 번째 원소 출력
print(train_lists_small[0])
print(test_lists_small[0])

(2, "I LOVED this movie! I am biased seeing as I am a huge Disney fan, but I really enjoyed myself. The action takes off running in the beginning of the film and just keeps going! This is a bit of a departure for Disney, they don't spend quite as much time on character development (my husband pointed this out)and there are no musical numbers. It is strictly action adventure. I thoroughly enjoyed it and recommend it to anyone who loves Disney, be they young or old.")
(1, 'This was an abysmal show. In short it was about this kid called Doug who guilt-tripped a lot. Seriously he could feel guilty over killing a fly then feeling guilty over feeling guilty for killing the fly and so forth. The animation was grating and unpleasant and the jokes cheap. <br /><br />It aired here in Sweden as a part of the "Disney time" show and i remember liking it some what but then i turned 13.<br /><br />I never got why some of the characters were green and purple too. What was up with that? <br /><br />Tru

In [3]:
train_texts = []
train_labels = []

In [4]:
for label, text in train_lists_small:
  # IMDB 데이터의 기준 레이블 2를 1 변경, 기본 레이블 1을 0으로 변경
  train_labels.append(1 if label==2 else 0)
  train_texts.append(text)

# test도 동일시
test_texts = []
test_labels = []

for label, text in test_lists_small:
  # IMDB 데이터의 기준 레이블 2를 1 변경, 기본 레이블 1을 0으로 변경
  test_labels.append(1 if label==2 else 0)
  test_texts.append(text)
  

print(train_texts[0])
print(train_labels[0])
print(test_texts[0])
print(test_labels[0])

I LOVED this movie! I am biased seeing as I am a huge Disney fan, but I really enjoyed myself. The action takes off running in the beginning of the film and just keeps going! This is a bit of a departure for Disney, they don't spend quite as much time on character development (my husband pointed this out)and there are no musical numbers. It is strictly action adventure. I thoroughly enjoyed it and recommend it to anyone who loves Disney, be they young or old.
1
This was an abysmal show. In short it was about this kid called Doug who guilt-tripped a lot. Seriously he could feel guilty over killing a fly then feeling guilty over feeling guilty for killing the fly and so forth. The animation was grating and unpleasant and the jokes cheap. <br /><br />It aired here in Sweden as a part of the "Disney time" show and i remember liking it some what but then i turned 13.<br /><br />I never got why some of the characters were green and purple too. What was up with that? <br /><br />Truly a horri

In [5]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=3)

print(len(train_texts))
print(len(train_labels))
print(len(val_texts))
print(len(val_labels))

800
800
200
200


### 토큰은 입력 텍스트의 기본 구성 요소를 의미하며 일반적으로단어를 지칭. 토크나이징은 입력 텍스트를 단어로 구분해 내는 과정. 토크나이징 결과를 딥러닝 모델이 사용하기 위해서는 잘게 구분된 토큰(텍스트 형태)을 숫자로 전환해야 함. 이 과정을 인코딩

In [6]:
# distilbert-base-uncased 모델에서 토크나이저 불러오기
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [7]:
# 토크나이저 실행
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True) # truncation은 모델의 디폴트 max_length를 넘는 입력 부분은 더 이상 받지 않고 절단

# 0번째 입력문(텍스트)의 5번째 토큰까지의 input_ids 출력
print(train_encodings["input_ids"][0][:5])

# 위의 결과를 디코딩하여 출력
print(tokenizer.decode(train_encodings["input_ids"][0][:5]))

[101, 4937, 11350, 2038, 2048]
[CLS] cat soup has two


In [8]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
  
  # 생성자 __init__()
  # 자신을 가리키는 매개변수 self 포함
  # 변수를 저장하기 위해 self.변수명을 사용
  def __init__(self, encodings, labels):
    self.encodings = encodings 
    self.labels = labels
  
  # 자신을 가리키는 매개변수 self 포함
  def __getitem__(self, idx):
    # self.encoding에 담긴 키(key)와 키값(value)을 items()로 추출
    # 이 값을 key와 val 변수에 담아 새로운 키(key)와
    # 키값을 갖는 딕셔너리 생성
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  
  # 자신을 가리키는 매개변수 self 포함
  def __len__(self):
    return len(self.labels)
  
train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)
  

In [9]:
for i in train_dataset:
  print(i)
  break

{'input_ids': tensor([  101,  4937, 11350,  2038,  2048,  1000,  7592, 14433,  1000,  1011,
         2828, 18401,  2015, 28866,  2075,  2006,  1037, 13576,  4440,  2083,
         1996, 25115,  1010,  2073,  2505,  2064,  4148,  1010,  1998,  2515,
         1012,  2023,  2568,  1011,  4440,  4691,  4004,  2460,  3594,  2053,
        13764,  8649,  1010,  4942, 21532,  2773, 22163,  2612,  1012,  2045,
         2003,  2053,  2126,  1997,  7851,  2023, 17183, 14088,  9476,  3272,
         2000,  2425,  2017,  2000,  2156,  2009,  2005,  4426,  1012,  1998,
         2191,  2469,  2053,  2028,  2104,  2184,  2003,  1999,  1996,  2282,
         1012,  4487,  6491,  6633,  5677,  3672,  1998,  2064,  3490, 10264,
         2964,  1998, 18186,  1998,  9576,  2854,  1998,  5573,  2331,  1998,
         2655,  3560, 27770,  2005,  2500,  2024,  2691,  6991,  1012,  7481,
         1012,  3383,  1996,  2087, 13432,  3746,  2003,  2008,  1997,  2019,
        10777,  3605,  1997,  2300,  2008,  1996, 

In [10]:
from transformers import DistilBertForSequenceClassification

# distilbert 모델 불러오기 
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='./results',
  num_train_epochs=8,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=64,
  warmup_steps=500,
  weight_decay=0.01,
  logging_dir='./logs',
  logging_steps=10,
)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [13]:
input_tokens = tokenizer(["I feel fantastic", "My life is going something wrong", "I have not figured out what the chosen title has to do with the movie."], truncation=True, padding=True)

outputs = model(torch.tensor(input_tokens['input_ids']).to(device))

label_dict = {0: 'positive', 1:'negative'}

print([label_dict[i] for i in torch.argmax(outputs['logits'], axis=1).cpu().numpy()])

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


['positive', 'positive', 'positive']


In [14]:
device

device(type='cuda')

In [15]:
from transformers import Trainer
trainer = Trainer(
  model = model,
  args = training_args,
  train_dataset = train_dataset,
  eval_dataset = val_dataset
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: viitamin-data (level2_cv4_dc). Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.695700
20,0.696400
30,0.694400
40,0.695000
50,0.686100
60,0.683600
70,0.671800
80,0.645300
90,0.615200
100,0.526900


TrainOutput(global_step=400, training_loss=0.2863156864605844, metrics={'train_runtime': 114.176, 'train_samples_per_second': 56.054, 'train_steps_per_second': 3.503, 'total_flos': 847791351398400.0, 'train_loss': 0.2863156864605844, 'epoch': 8.0})

In [16]:
input_tokens = tokenizer(["I feel fantastic", "My life is going something wrong", "I have not figured out what the chosen title has to do with the movie."], truncation=True, padding=True)

outputs = model(torch.tensor(input_tokens['input_ids']).to(device))

label_dict = {0: 'positive', 1:'negative'}

print([label_dict[i] for i in torch.argmax(outputs['logits'], axis=1).cpu().numpy()])

['negative', 'positive', 'positive']
